<a href="https://colab.research.google.com/github/snoopies/EEGTest/blob/main/csp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
import ipdb
print(__doc__)

# #################################################################################################################
# # Set parameters and read data

# avoid classification of evoked responses by using epochs that start 1s after cue onset.
tmin, tmax = -1., 4.                            #设置参数，记录点的前1秒后4秒用于生成epoch数据
event_id = dict(hands=2, feet=3)                #设置事件的映射关系
subject = 1
runs = [6, 10, 14]                              # motor imagery: hands vs feet
ipdb.set_trace() # Breakpoint

raw_fnames = eegbci.load_data(subject, runs)    # 获取想要读取的文件名称，这个应该是没有会默认下载的数据


Automatically created module for IPython interactive environment
--Return--
None
> /tmp/ipython-input-1437405622.py(25)<cell line: 0>()
     24 runs = [6, 10, 14]                              # motor imagery: hands vs feet
---> 25 ipdb.set_trace() # Breakpoint
     26 

ipdb> pp runs
[6, 10, 14]
